In [ ]:
# imports
import kagglehub
import numpy as np
import pandas as pd

# scikit-learn stuff
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE


# download the dataset
path = kagglehub.dataset_download("mlg-ulb/creditcardfraud")

print("path to dataset files:", path)

In [ ]:
import pandas as pd

# Trying to get a better sense of the data
csv_path = path + '/creditcard.csv'
df = pd.read_csv(csv_path)
df.head()

In [ ]:
df.describe()

In [ ]:
# split the dataset into training and test, use SMOTE since the dataset is imbalanced

X = df.iloc[:, :30] # all rows, first 30 columns
y = df.iloc[:, 30]  # all rows, last column

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=0
)

sm = SMOTE(random_state=2)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)

In [ ]:
# define the model and fit it on the training data
from sklearn.linear_model import RidgeClassifier

rc = RidgeClassifier(alpha=1, solver='sag') # decided on saga for the large number of features and samples
clf = rc.fit(X_train_res, y_train_res)
clf.score(X_test, y_test)


So, this score is good, but too good to be true without much work. I used SMOTE to balance the dataset, which was a good choice I believe, but the choice of model could be slightly better. Models like random forest, XGBoost, or a NN may capture non-linear patterns better. Additionally, accuracy is not the only measure of performance, especially on such an imbalanced dataset. 

The next step is to use better evaluation metrics. Going to fit two other models, and display more informative metrics like precision, recall, and F1-score for the minority class.

In [ ]:
from sklearn.metrics import classification_report

y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred)

Looking at the results, the model actually fails about 25% of the fraudulent cases. With only a 35% precision, which could be costly in something fragile like fraud detection. 

Trying a random forest classifier:

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier()
rf_clf.fit(X_train_res, y_train_res)
rf_clf.score(X_test, y_test)

In [ ]:
y_rf_pred = rf_clf.predict(X_test)
print(classification_report(y_test, y_rf_pred))

Pretty good results,  can see that 90% of cases that were classified as fraud were indeed fraudulent cases, but a slightly problematic recall of 82%. Now I want to try a neural net trained on the SMOTE dataset. 

In [ ]:
# had to use tf-nightly since this venv is on python 3.13
import keras
import tensorflow as tf
from keras import backend as K
from keras.models import Sequential
from keras.layers import Activation
from keras.layers import Dense, Dropout, BatchNormalization
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from sklearn.metrics import classification_report

In [ ]:
n_inputs = X_train_res.shape[1]

smote_model = Sequential([
    Dense(128, input_shape=(n_inputs,), activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

smote_model.summary()

smote_model.compile(optimizer=Adam(learning_rate=0.0005),
                    loss='binary_crossentropy',
                    metrics=['accuracy'])

In [ ]:
smote_model.fit(X_train_res, y_train_res,
                epochs=20,
                batch_size=64,
                validation_split=0.2,
                shuffle=True,
                verbose=1)

In [ ]:
y_pred_prob = smote_model.predict(X_test, verbose=1)
y_pred = (y_pred_prob > 0.5).astype(int)

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
# Trying to scale the data to improve performance
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_res_scaled = scaler.fit_transform(X_train_res)
X_test_scaled = scaler.transform(X_test)

smote_model.fit(X_train_res_scaled, y_train_res, 
                epochs=20,
                batch_size=32,
                validation_split=0.2,
                verbose=1)

In [ ]:
y_pred_prob_scaled = smote_model.predict(X_test_scaled, verbose=1)
y_pred = (y_pred_prob_scaled > 0.5).astype(int)
print(classification_report(y_test, y_pred))

In [ ]:
# Now try balancing the loss to penalize false negatives
from sklearn.utils.class_weight import compute_class_weight

class_weights = compute_class_weight('balanced', classes=np.unique(y_train_res), y=y_train_res)
class_weights_dict = dict(enumerate(class_weights))
smote_model.fit(X_train_res_scaled, y_train_res,
                epochs=20,
                batch_size=32,
                validation_split=0.2,
                class_weight=class_weights_dict,
                callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)],
                verbose=1)

In [ ]:
y_pred_prob = smote_model.predict(X_test_scaled, verbose=1)
y_pred = (y_pred_prob > 0.6).astype(int)
print(classification_report(y_test, y_pred))

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

smote_matrix = confusion_matrix(y_test, y_pred, labels=[0, 1])

disp = ConfusionMatrixDisplay(smote_matrix, display_labels=['No Fraud', 'Fraud'])
disp.plot()
plt.show()